In [1]:
import gym
import network
import os
import torch
import config as C
import numpy as np

In [2]:
env = gym.make('Pendulum-v0')

observation_space_size = env.observation_space.shape[0]
action_space_size = env.action_space.shape[0]

In [3]:
cwd = os.getcwd()
parameter_file = 'pendulum_nn_trained_model.pt'
cwd = os.path.join(cwd,parameter_file)

device = torch.device('cpu')
model = network.Agent(observation_space_size=observation_space_size,
                           action_space_size=action_space_size,
                           hidden_size=C.hidden_size)
model.load_state_dict(torch.load(cwd, map_location=device))

RuntimeError: Error(s) in loading state_dict for Agent:
	size mismatch for net.0.weight: copying a param with shape torch.Size([256, 6]) from checkpoint, the shape in current model is torch.Size([256, 3]).

In [ ]:
model.parameters

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

In [ ]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp
get_n_params(model)

In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

In [ ]:
device = 'cpu'
rewards = []

state = env.reset()

while True:
    #env.render()
    
    action = model(torch.tensor(state).float().unsqueeze(dim=0).to(device))
    
    #print('action is',action.cpu().item())
    
    # take the chosen action, observe the reward and the next state
    state, reward, done, _ = env.step(action=[action.cpu().item()])
    
    rewards.append(reward)
    
    if done:
        break

In [ ]:
np.sum(rewards)

In [ ]:
goal = []

for _ in range(0,100):
    
    rewards = []
    
    
    state = env.reset()
    while True:
        #env.render()

        action = model(torch.tensor(state).float().unsqueeze(dim=0).to(device))

        #print('action is',action.cpu().item())

        # take the chosen action, observe the reward and the next state
        state, reward, done, _ = env.step(action=[action.cpu().item()])

        rewards.append(reward)

        if done:
            break
    
    goal.append(np.sum(rewards))

In [ ]:
import matplotlib.pyplot as plt
plt.hist(goal,bins = 100)
plt.show()

In [ ]:
np.mean(goal)

In [ ]:
np.std(goal)